<a href="https://colab.research.google.com/github/Kkuntal990/CTW_2020/blob/master/Siamese2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import psutil
def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"zz
        bytes /= factor
print("="*40, "Memory Information", "="*40)
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}") ; print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}") ; print(f"Percentage: {svmem.percent}%")

======================================== Memory Information ========================================
Total: 25.51GB
Available: 24.54GB
Used: 639.60MB
Percentage: 3.8%


In [2]:
import torchvision
import torch
from torchsummary import summary
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
import os
from torchvision.utils import save_image
from torch.autograd import Variable

In [3]:
torch.cuda.empty_cache()

! nvidia-smi

Fri Jul 10 08:47:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
import h5py
f = h5py.File("/content/gdrive/My Drive/CTW2020/Processed Data/udata2020.hdf5","r")
import numpy as np
_ , key = next(enumerate(f.keys()))
X = f[key][:]
f.close()

In [6]:
X.shape

(36192, 56, 924, 2)

In [7]:
X = np.sqrt(X[:,:,:,0]**2 + X[:,:,:,1]**2)

In [8]:
import h5py
f = h5py.File("/content/gdrive/My Drive/CTW2020/Processed Data/Labelled_1.hdf5","r")

H = f["H_Est"][:]
Pos = f["Pos"][:]
SNR = f["SNR"][:]
f.close()

print(H.shape)
print(SNR.shape)

(4979, 56, 924, 2)
(4979, 56)


In [9]:
H = np.sqrt(H[:,:,:,0]**2 + H[:,:,:,1]**2)

In [10]:
from sklearn.model_selection import train_test_split
H_Train, H_Test , Pos_Train , Pos_Test = train_test_split(H,Pos,test_size=0.1, random_state=42)

In [11]:
H_Train, H_Val , Pos_Train , Pos_Val = train_test_split(H_Train,Pos_Train,test_size=0.1, random_state=99)

In [12]:
print(H_Train.shape, H_Val.shape, H_Test.shape)


(4032, 56, 924) (449, 56, 924) (498, 56, 924)


In [13]:
H_Train = torch.tensor(H_Train,dtype=torch.float)
Pos_Train = torch.tensor(Pos_Train,dtype=torch.float)
H_Val = torch.tensor(H_Val,dtype=torch.float)
Pos_Val = torch.tensor(Pos_Val,dtype=torch.float)

In [15]:
class Sammon(nn.Module):
    def __init__(self):
        super(Sammon, self).__init__()
        
        
        self.main = nn.Sequential(
            nn.Conv2d(1,2, (1,8), stride=(1,2)),
            nn.ReLU(),
            nn.Conv2d(2, 4, (1,8), stride=(1,4)),
            nn.ReLU())
        
        self.Project = nn.Sequential(nn.Linear(25312,12000),
                                     nn.Tanh(),
                                     nn.Dropout(p=0.2),
                                     nn.Linear(12000,7500),
                                     nn.Softplus(),
                                     nn.Dropout(p=0.2),
                                     nn.Linear(7500,4500),
                                     nn.Tanh(),
                                     nn.Dropout(p=0.2),
                                     nn.Linear(4500,1024),
                                     nn.Softplus(),
                                     nn.Dropout(p=0.2),
                                     nn.Linear(1024,256),
                                     nn.Tanh(),
                                     nn.Dropout(p=0.2),
                                     nn.Linear(256,128),
                                     nn.Tanh(),
                                     nn.Dropout(p=0.1),
                                     nn.Linear(128,64),
                                     nn.Tanh(),
                                     nn.Dropout(p=0.1),
                                     nn.Linear(64,16),
                                     nn.Tanh(),
                                     nn.Dropout(p=0.1),
                                     nn.Linear(16,8),
                                     nn.Tanh(),
                                     nn.Linear(8,3))
        self.alpha = nn.Parameter(torch.rand(1))
                                     
          
            
    def forward(self, input):
        return (self.aplha)*(self.Project(self.main(input).view(-1,25312)).view(-1,3))





In [16]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
sammon = Sammon().to(device)
#reg = Regressor().to(device)

In [17]:
opt = optim.Adam([{'params': model.main.parameters()},{'params': model.Project.parameters()},{'params': model.alpha.parameters(),'lr':1e-2}], lr=1e-4)
MSE = nn.MSELoss()

In [18]:
summary(sammon, (1, 56,924))
summary(reg, (4500,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 2, 56, 459]              18
              ReLU-2           [-1, 2, 56, 459]               0
            Conv2d-3           [-1, 4, 56, 113]              68
              ReLU-4           [-1, 4, 56, 113]               0
            Linear-5                [-1, 12000]     303,756,000
              Tanh-6                [-1, 12000]               0
            Linear-7                 [-1, 7500]      90,007,500
          Softplus-8                 [-1, 7500]               0
            Linear-9                 [-1, 4500]      33,754,500
             Tanh-10                 [-1, 4500]               0
Total params: 427,518,086
Trainable params: 427,518,086
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.20
Forward/backward pass size (MB): 1.54
Params size (MB): 1630.85


In [19]:
A = list(range(len(X)))

In [ ]:
def process(indices):
    # data of size 500

    pairs = []
    for i in range(len(indices)):
        for j in range(i+1,len(indices)):
            pairs.append([i,j])

    return pairs


import random
def chunk(xs, n):
    ys = list(xs)
    random.shuffle(ys)
    chunk_length = len(ys) // n
    needs_extra = len(ys) % n
    start = 0
    for i in range(n):
        if i < needs_extra:
            end = start + chunk_length + 1
        else:
            end = start + chunk_length
        yield ys[start:end]
        start = end


In [21]:
pairs.shape

(72384, 2)

In [22]:
N_b2 = len(H_Train)
N_b = 72
N_val = len(H_Val)
bs1= 512
bs2 = 8
epsilon = 1e-6

print(N_b2, N_b, N_val)

4032 72384 449


In [23]:
X.shape

(36192, 56, 924)

In [24]:
def custom_loss(XX, YY):
  return torch.sum((1/XX)*(torch.square(XX-YY)))



In [25]:
for i in range(100):
  batch1=0
  batch2=0
  for ch in chunks(A,72):
    pairs = process(ch)
    for j in range(0,len(pairs),bs1):
      sammon.zero_grad()
      end = min(j+bs1,len(pairs))
      Xn,Xm = torch.tensor(X[pairs[j:end,0]],dtype=torch.float),  torch.tensor(X[pairs[j:end,1]],dtype=torch.float)
      Xn , Xm = Variable(Xn, requires_grad=True) , Variable(Xm, requires_grad=True)
      Xn = Xn.to(device)
      Xm = Xm.to(device)
      Yn = sammon(Xn)
      Ym = sammon(Xm)
      
      XX = torch.norm((Xn-Xm),p=2, dim = [1,2])
      YY = torch.norm((Yn-Ym),p=2, dim = 1)
      XX = torch.sqrt(torch.square(XX) + epsilon)
      YY = torch.sqrt(torch.square(YY) + epsilon)
      loss = custom_loss(XX, YY)
      #print(loss)
      loss.backward()
      opt.step()


    batch1+=1
    print("Batch",batch1,"/",(len(A)//72)+1,"Sammon Loss :" , loss.item())

  for k in range(0,N_b2,bs2):
    end = min(k+bs2,N_b2)
    H1 = H_Train[k:end]
    sammon.zero_grad()
    reg.zero_grad()
    H1 = Variable(H1,requires_grad=True)
    H1 = H1.to(device)
    Y1 = reg(sammon(H1.view(-1,1,56,924)))
    loss_1 = MSE(Y1,Pos_Train[k:end].to(device)).view(-1)
    loss_1.backward()
    opt1.step()
    opt2.step()
    batch2+=1
    if(batch2%10 == 0):
      print("Batch",batch2,"/",(N_b2//bs2)+1,"Regressor Loss :" , loss_1.item())
  print("Epoch ",i,"/",100," Sammon Loss :",loss.item() ," Regressor Loss :" , loss_1.item())
  for l in range(N_val):
    H_val = torch.tensor(H_Val[l],dtype=torch.float)
    H_val = H_val.to(device)
    with torch.no_grad():
      Y_Val = reg(sammon(H_val.view(-1,1,56,924)))
      loss_Val = MSE(Y_Val,Pos_Val[l].to(device)).view(-1)
  print("Val Loss :" , loss_Val.item())

    

      
  


Batch 10 / 1132 Sammon Loss : 1895.8427734375
Batch 20 / 1132 Sammon Loss : 1176.537353515625
Batch 30 / 1132 Sammon Loss : 1350.989501953125
Batch 40 / 1132 Sammon Loss : 1379.025146484375
Batch 50 / 1132 Sammon Loss : 1583.758056640625
Batch 60 / 1132 Sammon Loss : 764.25634765625
Batch 70 / 1132 Sammon Loss : 629.0015869140625
Batch 80 / 1132 Sammon Loss : 591.71240234375
Batch 90 / 1132 Sammon Loss : 598.542724609375
Batch 100 / 1132 Sammon Loss : 461.2624816894531
Batch 110 / 1132 Sammon Loss : 264.6654357910156
Batch 120 / 1132 Sammon Loss : 155.77392578125
Batch 130 / 1132 Sammon Loss : 419.4209899902344
Batch 140 / 1132 Sammon Loss : 705.1890258789062
Batch 150 / 1132 Sammon Loss : 582.5146484375
Batch 160 / 1132 Sammon Loss : 469.21356201171875
Batch 170 / 1132 Sammon Loss : 1361.4617919921875
Batch 180 / 1132 Sammon Loss : 1397.240966796875
Batch 190 / 1132 Sammon Loss : 1167.712890625
Batch 200 / 1132 Sammon Loss : 1064.260986328125
Batch 210 / 1132 Sammon Loss : 1087.994140

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([1, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Streaming output truncated to the last 5000 lines.
Batch 690 / 1132 Sammon Loss : 1096.6151123046875
Batch 700 / 1132 Sammon Loss : 1517.276611328125
Batch 710 / 1132 Sammon Loss : 1067.60107421875
Batch 720 / 1132 Sammon Loss : 1655.429931640625
Batch 730 / 1132 Sammon Loss : 700.5896606445312
Batch 740 / 1132 Sammon Loss : 1344.64501953125
Batch 750 / 1132 Sammon Loss : 1028.8076171875
Batch 760 / 1132 Sammon Loss : 1475.412353515625
Batch 770 / 1132 Sammon Loss : 2211.4658203125
Batch 780 / 1132 Sammon Loss : 1908.04638671875
Batch 790 / 1132 Sammon Loss : 1220.09521484375
Batch 800 / 1132 Sammon Loss : 2269.52392578125
Batch 810 / 1132 Sammon Loss : 1015.3395385742188
Batch 820 / 1132 Sammon Loss : 1203.2666015625
Batch 830 / 1132 Sammon Loss : 2103.70556640625
Batch 840 / 1132 Sammon Loss : 1715.426025390625
Batch 850 / 1132 Sammon Loss : 3473.804931640625
Batch 860 / 1132 Sammon Loss : 3790.21875
Batch 870 / 1132 Sammon Loss : 4177.44921875
Batch 880 / 1132 Sammon Loss : 1763.414

KeyboardInterrupt: ignored

In [ ]:
pairs[2:9,0]

array([2, 3, 4, 5, 6, 7, 8])